Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [27]:
import pandas as pd
import geopandas
cross_lochs = pd.read_csv('../cross_lochs_gdf_pft.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf.head(2)

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
0,0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,...,0.072792,0.075150,0.074088,0.074698,0.072248,1,POINT (444724.308 6472605.655),0.001099,calluna_mix,1
1,1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,...,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.308 6472605.655),0.003365,calluna_mix,1


In [28]:
# percentage = round(len(cross_lochs_gdf)/100*70) 
# train = cross_lochs_gdf.head(percentage)  
# test = cross_lochs_gdf.iloc[percentage:len(cross_lochs_gdf),:]

from sklearn.model_selection import train_test_split

train, test = train_test_split(cross_lochs_gdf, test_size=0.3)

train.head()

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
14992,14992,0.000934,0.002335,0.003164,0.004591,0.004467,0.006978,0.008833,0.010817,0.013312,...,0.064559,0.065331,0.063870,0.064325,0.061810,14993,POINT (445589.308 6472240.655),0.000934,shrub_sphagnum,74
26593,26593,0.002531,-0.000137,0.004182,0.006113,0.007100,0.009647,0.011002,0.012171,0.015218,...,0.088955,0.088454,0.090158,0.086885,0.089412,26594,POINT (444724.308 6471950.655),0.002531,shrub_sphagnum,132
11167,11167,0.002886,0.003252,0.005460,0.007206,0.007818,0.011227,0.013430,0.014060,0.017475,...,0.100040,0.102975,0.101189,0.100309,0.099879,11168,POINT (444734.308 6472330.655),0.002886,shrub_sphagnum,56
27401,27401,0.001448,0.002440,0.004668,0.005609,0.006491,0.008425,0.010274,0.011574,0.013661,...,0.062082,0.062946,0.061086,0.060301,0.061506,27402,POINT (445719.308 6471935.655),0.001448,rushes_sedges,135
447,447,0.002176,0.002448,0.004856,0.006185,0.007793,0.010245,0.013715,0.015758,0.017584,...,0.108418,0.105762,0.110234,0.109437,0.109742,448,POINT (444929.308 6472595.655),0.002176,shrub_sphagnum,3


In [29]:
x_train = train[train.columns[1:359]]
y_train = train[train.columns[362]]

x_test = test[test.columns[1:359]]
y_test = test[test.columns[362]]

x_test.head(2)

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
18182,0.000966,0.002393,0.004038,0.005373,0.005853,0.008410,0.011335,0.013074,0.014832,0.016514,...,0.091133,0.091739,0.090761,0.092695,0.090972,0.091643,0.093351,0.090977,0.091653,0.092302
2373,0.001633,0.002376,0.003934,0.004007,0.004950,0.006804,0.009719,0.010944,0.013236,0.014713,...,0.076503,0.075954,0.074826,0.072765,0.075480,0.070757,0.072072,0.071117,0.072036,0.069758


In [30]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


Baseline Accuracy: 0.7823431426985008


In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=2, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.876011423131842
0.89 accuracy with a standard deviation of 0.01


In [32]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

0.88 accuracy with a standard deviation of 0.00


In [33]:
model_results = test[test.columns[360:363]]

model_results.head()

,geometry,GRID_CODE,PFT
18182,POINT (445299.308 6472160.655),0.000966,shrub_sphagnum
2373,POINT (445424.308 6472550.655),0.001633,sphagnum_r
34418,POINT (445279.308 6471760.655),0.000707,shrub_sphagnum
19857,POINT (445554.308 6472120.655),0.000800,water
3715,POINT (445029.308 6472515.655),0.001764,shrub_sphagnum


In [34]:
model_results = model_results.assign(clf_pred = clf_pred)

model_results.head()

,geometry,GRID_CODE,PFT,clf_pred
18182,POINT (445299.308 6472160.655),0.000966,shrub_sphagnum,shrub_sphagnum
2373,POINT (445424.308 6472550.655),0.001633,sphagnum_r,shrub_sphagnum
34418,POINT (445279.308 6471760.655),0.000707,shrub_sphagnum,shrub_sphagnum
19857,POINT (445554.308 6472120.655),0.000800,water,water
3715,POINT (445029.308 6472515.655),0.001764,shrub_sphagnum,shrub_sphagnum


In [35]:
train_focused = train[train.columns[360:363]]

train_focused.head()

,geometry,GRID_CODE,PFT
14992,POINT (445589.308 6472240.655),0.000934,shrub_sphagnum
26593,POINT (444724.308 6471950.655),0.002531,shrub_sphagnum
11167,POINT (444734.308 6472330.655),0.002886,shrub_sphagnum
27401,POINT (445719.308 6471935.655),0.001448,rushes_sedges
447,POINT (444929.308 6472595.655),0.002176,shrub_sphagnum


In [39]:
# model_results.to_file("cross_lochs_results_gdf.shp")

In [40]:
# train_focused.to_file("cross_lochs_train_gdf.shp")

In [42]:
# model_results.to_csv("cross_lochs_results_gdf.csv")
# train_focused.to_csv("cross_lochs_train_gdf.csv")

In [53]:
cross_lochs_rf_gdf = model_results.append(train_focused)
cross_lochs_rf_gdf

,geometry,GRID_CODE,PFT,clf_pred
18182,POINT (445299.308 6472160.655),0.000966,shrub_sphagnum,shrub_sphagnum
2373,POINT (445424.308 6472550.655),0.001633,sphagnum_r,shrub_sphagnum
34418,POINT (445279.308 6471760.655),0.000707,shrub_sphagnum,shrub_sphagnum
19857,POINT (445554.308 6472120.655),0.000800,water,water
3715,POINT (445029.308 6472515.655),0.001764,shrub_sphagnum,shrub_sphagnum
...,...,...,...,...
32367,POINT (445174.308 6471810.655),0.000848,shrub_sphagnum,NaN
38701,POINT (445379.308 6471655.655),0.000024,shrub_sphagnum,NaN
33314,POINT (444834.308 6471785.655),0.001136,shrub_sphagnum,NaN
2028,POINT (445729.308 6472560.655),0.001496,shrub_sphagnum,NaN


In [55]:
cross_lochs_rf_gfd["difference"] = ""

for index, row in cross_lochs_rf_gdf.iterrows():
    if cross_lochs_rf_gdf.iloc[index, 2] == cross_lochs_rf_gdf.iloc[index, 3]:
        cross_lochs_rf_gdf.iloc[index, 4] = 'Same'
    if cross_lochs_rf_gdf.iloc[index, 3] == "NaN":
        cross_lochs_rf_gdf.iloc[index, 4] = 'Same'
    else:
        cross_lochs_rf_gdf.iloc[index, 4] = 'Different'

In [56]:
cross_lochs_rf_gdf.head()

,geometry,GRID_CODE,PFT,clf_pred,difference
18182,POINT (445299.308 6472160.655),0.000966,shrub_sphagnum,shrub_sphagnum,Different
2373,POINT (445424.308 6472550.655),0.001633,sphagnum_r,shrub_sphagnum,Different
34418,POINT (445279.308 6471760.655),0.000707,shrub_sphagnum,shrub_sphagnum,Different
19857,POINT (445554.308 6472120.655),0.000800,water,water,Different
3715,POINT (445029.308 6472515.655),0.001764,shrub_sphagnum,shrub_sphagnum,Different


In [ ]:
# return to - not workedin difference column